<a href="https://colab.research.google.com/github/roytravel/machine-learning/blob/master/start_tf_advance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 텐서플로우 라이브러리 import

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-rc1
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

     |████████████████████████████████| 380.5MB 40kB/s 
     |████████████████████████████████| 4.3MB 46.8MB/s 
     |████████████████████████████████| 501kB 48.5MB/s 


## MNIST 데이터셋 로드




In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 채널 차원을 추가합니다.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

## 데이터셋 셔플 및 배치 생성

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 케라스의 모델 서브클래싱 API 사용, tf.keras 모델 생성

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation = 'softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

## 훈련에 필요한 옵티마이저와 손실 함수를 선택

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

## 모델 손실과 성능을 측정할 지표 선택. 
## 에포크가 진행되는 동안 수집된 측정 지표를 바탕으로 최종결과를 출력

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accruacy')

train_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## tf.GradientTape를 사용하여 모델을 훈련

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

## 모델 테스트

In [0]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [23]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

에포크: 1, 손실: 0.13358508050441742, 정확도: 96.04000091552734, 테스트 손실: 0.06433883309364319, 테스트 정확도: 97.89999389648438
에포크: 2, 손실: 0.08849693089723587, 정확도: 97.34749603271484, 테스트 손실: 0.06332173198461533, 테스트 정확도: 97.97000122070312
에포크: 3, 손실: 0.06612986326217651, 정확도: 98.00721740722656, 테스트 손실: 0.06286251544952393, 테스트 정확도: 98.00666809082031
에포크: 4, 손실: 0.05301520600914955, 정확도: 98.39291381835938, 테스트 손실: 0.06298697739839554, 테스트 정확도: 98.06749725341797
에포크: 5, 손실: 0.04406382143497467, 정확도: 98.65533447265625,